# <font color='orange'>Lab | Random Forests</font>

- Instructions
- Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.
- Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.

In [148]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
numerical=pd.read_csv('numerical.csv')
categorical=pd.read_csv('categorical.csv')
targets=pd.read_csv('target.csv')

In [134]:
data=pd.concat([numerical, categorical], axis=1)
data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,0,60.000000,5,9,0,0,39,34,18,10,...,37,12,92,8,94,2,95,12,89,11
1,1,46.000000,6,9,16,0,15,55,11,6,...,52,2,93,10,95,12,95,12,93,10
2,1,61.611649,3,1,2,0,20,29,33,6,...,0,2,91,11,92,7,95,12,90,1
3,0,70.000000,1,4,2,0,23,14,31,3,...,28,1,87,11,94,11,95,12,87,2
4,0,78.000000,3,2,60,1,28,9,53,26,...,20,1,93,10,96,1,96,1,79,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,0,2,96,2,96,2,96,2,96,2
95408,1,48.000000,7,9,1,0,31,43,19,4,...,50,1,96,3,96,3,96,3,96,3
95409,1,60.000000,5,9,0,0,18,46,20,7,...,38,1,96,3,95,1,96,10,94,10
95410,0,58.000000,7,9,0,0,28,35,20,9,...,40,5,90,11,96,8,97,1,86,12


In [135]:
# Vamos a hacer una matriz de correlacion para eliminar las columnas con una correlacion alta
corr_matrix=data.corr()

In [136]:
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))

In [137]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.70)] # Eliminamos las columnas con una correlacion mas alta al 70%
print(to_drop)

['POP902', 'POP903', 'ETH8', 'ETH13', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906', 'AGEC6', 'AGEC7', 'CHILC1', 'CHILC2', 'CHILC3', 'CHILC5', 'HHAGE1', 'HHAGE2', 'HHAGE3', 'HHN1', 'HHN3', 'HHN4', 'HHN5', 'HHN6', 'MARR3', 'MARR4', 'HHP1', 'HHP2', 'DW2', 'DW5', 'DW6', 'DW8', 'HV2', 'HV3', 'HV4', 'HU1', 'HU2', 'HHD1', 'HHD2', 'HHD3', 'HHD4', 'HHD5', 'HHD6', 'HHD9', 'HHD10', 'HHD11', 'HHD12', 'ETHC1', 'ETHC2', 'ETHC3', 'ETHC4', 'ETHC5', 'ETHC6', 'HVP1', 'HVP2', 'HVP3', 'HVP4', 'HVP5', 'HVP6', 'HUR2', 'RHP1', 'RHP2', 'RHP3', 'HUPA1', 'HUPA2', 'HUPA5', 'HUPA6', 'HUPA7', 'RP1', 'RP2', 'RP3', 'RP4', 'IC1', 'IC2', 'IC3', 'IC4', 'IC5', 'IC11', 'IC12', 'IC14', 'IC15', 'IC16', 'IC17', 'IC18', 'IC19', 'IC20', 'IC21', 'IC22', 'IC23', 'HHAS1', 'HHAS3', 'HHAS4', 'TPE4', 'TPE11', 'TPE12', 'LFC2', 'LFC3', 'LFC4', 'LFC5', 'LFC7', 'OCC2', 'EIC1', 'EIC4', 'OEDC1', 'OEDC2', 'OEDC3', 'EC1', 'EC7', 'EC8', 'SEC4', 'AFC1', 'AFC2', 'AFC3', 'AFC5', 'VC1', 'VC3', 'LSC2', 'LSC3', 'VOC1', 'VOC2', 'VOC3', 'HC1', 

In [138]:
drop_list=list(to_drop)
drop_list

['POP902',
 'POP903',
 'ETH8',
 'ETH13',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW2',
 'DW5',
 'DW6',
 'DW8',
 'HV2',
 'HV3',
 'HV4',
 'HU1',
 'HU2',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP1',
 'HVP2',
 'HVP3',
 'HVP4',
 'HVP5',
 'HVP6',
 'HUR2',
 'RHP1',
 'RHP2',
 'RHP3',
 'HUPA1',
 'HUPA2',
 'HUPA5',
 'HUPA6',
 'HUPA7',
 'RP1',
 'RP2',
 'RP3',
 'RP4',
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC11',
 'IC12',
 'IC14',
 'IC15',
 'IC16',
 'IC17',
 'IC18',
 'IC19',
 'IC20',
 'IC21',
 'IC22',
 'IC23',
 'HHAS1',
 'HHAS3',
 'HHAS4',
 'TPE4',
 'TPE11',
 'TPE12',
 'LFC2',
 'LFC3',
 'LFC4',
 'LFC5',
 'LFC7',
 'OCC2',
 'EIC1',
 'EIC4',
 'OEDC1',
 'OEDC2',
 'OEDC3',
 'EC1',
 'EC7',
 '

In [139]:
data=data.drop(drop_list, axis=1)
data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_MM
0,0,60.000000,5,9,0,0,39,34,18,10,...,89,1,37,12,8,94,2,95,12,11
1,1,46.000000,6,9,16,0,15,55,11,6,...,94,1,52,2,10,95,12,95,12,10
2,1,61.611649,3,1,2,0,20,29,33,6,...,90,1,0,2,11,92,7,95,12,1
3,0,70.000000,1,4,2,0,23,14,31,3,...,87,1,28,1,11,94,11,95,12,2
4,0,78.000000,3,2,60,1,28,9,53,26,...,86,1,20,1,10,96,1,96,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,96,1,0,2,2,96,2,96,2,2
95408,1,48.000000,7,9,1,0,31,43,19,4,...,96,1,50,1,3,96,3,96,3,3
95409,1,60.000000,5,9,0,0,18,46,20,7,...,95,1,38,1,3,95,1,96,10,10
95410,0,58.000000,7,9,0,0,28,35,20,9,...,86,1,40,5,11,96,8,97,1,12


In [140]:
# ya no tenemos nulos ni columnas con una alta correlacion mayor a 70% 
# asi que vamos a transformar nuestra data
numerical = data.select_dtypes("number")
categorical = data.select_dtypes("object")


In [141]:
# Aplicamos label encoder a la categorica
categorical=categorical.apply(preprocessing.LabelEncoder().fit_transform)
categorical

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,3,0,0,0,1,2,3
1,0,0,1,0,3,0,2
2,7,1,1,0,1,2,1
3,0,1,0,0,1,2,1
4,1,0,0,0,2,0,2
...,...,...,...,...,...,...,...
95407,11,0,1,0,3,2,0
95408,8,0,1,0,2,0,0
95409,5,0,1,0,1,1,0
95410,0,0,0,0,2,0,0


In [142]:
# Aplicamos Standar Scaler a la numerica
Standard=StandardScaler()
Standard=Standard.fit_transform(numerical)
Standard=pd.DataFrame(Standard) 
Standard


,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,190
0,-0.056847,-1.115292e-01,0.509147,0.717411,-0.356881,-0.206977,0.745798,0.284659,-0.826571,0.719013,...,-0.699830,-0.02304,0.370442,3.012879,0.419080,-0.201169,-1.236357,-0.789204,1.257673,1.237697
1,-0.055799,-1.080356e+00,1.097566,0.717411,1.362283,-0.206977,-1.346527,1.675602,-1.221851,-0.191801,...,0.755973,-0.02304,1.049599,-0.290923,0.932504,0.374914,1.266242,-0.789204,1.257673,0.985761
2,-0.055799,1.267017e-10,-0.667692,-2.239375,-0.141985,-0.206977,-0.910626,-0.046518,0.020458,-0.191801,...,-0.408669,-0.02304,-1.304814,-0.290923,1.189216,-1.353334,0.014943,-0.789204,1.257673,-1.281661
3,-0.056847,5.804901e-01,-1.844530,-1.130581,-0.141985,-0.206977,-0.649086,-1.040049,-0.092480,-0.874911,...,-1.282151,-0.02304,-0.037053,-0.621303,1.189216,-0.201169,1.015982,-0.789204,1.257673,-1.029725
4,-0.056847,1.134106e+00,-0.667692,-1.869777,6.089983,-0.009563,-0.213185,-1.371225,1.149829,4.362269,...,-1.573311,-0.02304,-0.399270,-0.621303,0.932504,0.950996,-1.486617,1.115522,-1.336135,-0.777790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,-0.055799,1.267017e-10,0.509147,0.717411,-0.356881,2.556819,0.484257,1.145719,-1.221851,0.035903,...,1.338294,-0.02304,-1.304814,-0.290923,-1.121193,0.950996,-1.236357,1.115522,-1.100334,-1.029725
95408,-0.055799,-9.419524e-01,1.685985,0.717411,-0.249433,-0.206977,0.048356,0.880777,-0.770102,-0.647208,...,1.338294,-0.02304,0.959045,-0.621303,-0.864481,0.950996,-0.986097,1.115522,-0.864533,-0.777790
95409,-0.055799,-1.115292e-01,0.509147,0.717411,-0.356881,-0.206977,-1.084987,1.079483,-0.713634,0.035903,...,1.047134,-0.02304,0.415719,-0.621303,-0.864481,0.374914,-1.486617,1.115522,0.786072,0.985761
95410,-0.056847,-2.499331e-01,1.685985,0.717411,-0.356881,-0.206977,-0.213185,0.350894,-0.713634,0.491310,...,-1.573311,-0.02304,0.506273,0.700217,1.189216,0.950996,0.265203,3.020249,-1.336135,1.489632


In [143]:
# Vamos a trabajar con el Target b
X=np.concatenate([Standard,categorical], axis=1)
X=pd.DataFrame(X)
y1=targets['TARGET_B']


In [145]:
# Aplicar SMOTE PARA LA DATA Y TARGET 1
upsampling1 = SMOTE()
X_smote,y_smote = upsampling1.fit_resample(X,y1)
y_smote.value_counts()

0    90569
1    90569
Name: TARGET_B, dtype: int64

In [149]:
# Aplicamos Random Forest 
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.20, random_state=50)
model = DecisionTreeClassifier(random_state=50)
model.fit(X_train, y_train)
scores=cross_val_score(model,X_train, y_train, cv=10,n_jobs=-1)
y_pred=model.predict(X_test)
print("Train Score:", np.mean(scores))
print("Accuracy test:",metrics.accuracy_score(y_test, y_pred))


Train Score: 0.9354702919053205
Accuracy test: 0.9365684001324942
